In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")
!export OPENAI_API_KEY="4"


In [2]:
%env OPENAI_API_KEY="g"

env: OPENAI_API_KEY="g"


In [3]:
import os
import json
import seaborn as sns
from collections import Counter
import openai
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir
from games.ultimatum.ultimatum_multi_turn.game import MultiTurnUltimatumGame
import itertools
from collections import defaultdict
import os
import json
import openai
import numpy as np
import pandas as pd
from explorer.utils import load_states_from_dir

In [4]:
def get_from_summary(key, game_state, default=None):
    if key in game_state.game_state[-1]['summary']:
        return game_state.game_state[-1]['summary'][key]
    else:
        return default

def compute_game_summary(game_states):

    game_name = np.array([g.__class__.__name__ for g in game_states])[:, None]
    models = np.array([[ p.model for p in g.players] for g in game_states])
    outcomes = np.array([get_from_summary('player_outcome', g) for g in game_states])
    valuations = np.array([get_from_summary('player_valuation', g, default=[None,None])  for g in game_states])
    initial_resources = np.array([get_from_summary('initial_resources', g) for g in game_states])
    final_resources = np.array([get_from_summary('final_resources', g) for g in game_states]), 
    resources_delta = (final_resources-initial_resources)[0]
        
    resources_delta = np.array([v.value(r) if v else r.value() for r,v in zip(resources_delta.reshape(-1,), valuations.reshape(-1))])
    resources_delta = resources_delta.reshape(-1, 2)

    social_behavior = np.array([g.player_social_behaviour for g in game_states])
    social_behavior = social_behavior.reshape(-1, 2)

    
    df = np.concatenate((game_name, models, outcomes,  resources_delta, social_behavior), axis=1)
    df = pd.DataFrame(df, columns=['game_name','model_1','model_2','outcome_1','outcome_2', 'resource_delta_1', 'resource_delta_2', "social_1", "social_2"])

    return df
      


In [5]:
from glob import glob

In [6]:
def check_if_same_number_of_games(df_):
    
    c = df_["model_1"] + " ### " + df_["model_2"]
    counter = Counter(c)
    x = list(dict(counter).values())
    if x.count(x[0]) != len(x):
        print("MISSING GAMES")
    else:
        print(f"ALL GOOD, {x[0]} games for each pair")
    print()
    return counter


In [7]:
logs = ["ultimatum_multi_section_one"]

In [23]:
import os
for a in glob("../experiments/section_one/.logs/*"):
    log_dir = a
    game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 
    df = compute_game_summary(game_states)
    print(os.path.basename(a))
    check_if_same_number_of_games(df)
    

THERE ARE 240 log files
240 Loaded Successfully
ultimatum_multi_section_one
ALL GOOD, 20 games for each pair

THERE ARE 480 log files
480 Loaded Successfully
trading_section_one
ALL GOOD, 40 games for each pair

THERE ARE 384 log files
384 Loaded Successfully
buysell_section_one
ALL GOOD, 32 games for each pair



In [24]:
import os
for a in glob("../experiments/section_two/.logs/*"):
    log_dir = a
    game_states = load_states_from_dir(log_dir) #+ load_states_from_dir(log_dir) 
    df = compute_game_summary(game_states)
    print(os.path.basename(a))
    check_if_same_number_of_games(df)
    

THERE ARE 45 log files
45 Loaded Successfully
social_behaviors_buy_sell
ALL GOOD, 45 games for each pair

THERE ARE 600 log files
600 Loaded Successfully
social_behaviors_trading
ALL GOOD, 600 games for each pair

THERE ARE 60 log files
60 Loaded Successfully
social_behaviors_ultimatum
ALL GOOD, 60 games for each pair



In [18]:
game_states = load_states_from_dir("../experiments/section_one/.logs/trading_section_one/") #+ load_states_from_dir(log_dir) 
df = compute_game_summary(game_states)


Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1705266769385/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "/home/vinid/projects/negotiation/_notebooks/../explorer/utils.py", line 112, in load_states_from_dir
    game.game_state[-1]["current_iteration"] == "END"
AssertionError: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1705266769385/game_state.json has not ended



Exception Type: AssertionError
Exception Message: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1705266795283/game_state.json has not ended

Stack Trace:
Traceback (most recent call last):
  File "/home/vinid/projects/negotiation/_notebooks/../explorer/utils.py", line 112, in load_states_from_dir
    game.game_state[-1]["current_iteration"] == "END"
AssertionError: WARNING : Game  ../experiments/section_one/.logs/trading_section_one/1705266795283/game_state.json ha

In [19]:
c = df["social_1"] + " ### " + df["social_2"]
counter = Counter(c)
x = list(dict(counter).values())

In [20]:
x

[480]

In [17]:
counter

Counter({' ### ': 20,
         ' ### \nYou must fake being desperate. Supplicate and beg to get more dollars.': 20,
         ' ### \nYou are cunning and sly in your plan to get more than your opponent. Humiliate your opponent as much as possible. Insult them.': 19})

In [46]:
game_states = load_states_from_dir("../experiments/section_two/.logs/social_behaviors_ultimatum/") #+ load_states_from_dir(log_dir) 
df = compute_game_summary(game_states)

THERE ARE 60 log files
60 Loaded Successfully


In [47]:
df

,game_name,model_1,model_2,outcome_1,outcome_2,resource_delta_1,resource_delta_2,social_1,social_2
0,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 50,Dollars: 50,-50,50,,
1,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 60,Dollars: 40,-40,40,,
2,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 60,Dollars: 40,-40,40,,
3,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 60,Dollars: 40,-40,40,,
4,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 50,Dollars: 50,-50,50,,
5,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 50,Dollars: 50,-50,50,,
6,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 60,Dollars: 40,-40,40,,
7,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 60,Dollars: 40,-40,40,,
8,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 50,Dollars: 50,-50,50,,
9,MultiTurnUltimatumGame,gpt-4-1106-preview,gpt-4-1106-preview,Dollars: 50,Dollars: 50,-50,50,,
